In [12]:
import together 
import os
together.api_key="3aa940ae895a17febc136d71f381320187e4a727781bb83212df669bc850571d"

In [2]:
import pandas as pd
import json

In [3]:
df = pd.read_csv("/Users/anubhavnehru/Desktop/UT/Spring Sem/Deep Learning /Project/Resume Screening/cleaned_resume.csv")

In [4]:
df

,Category,Cleaned_Resume
0,Data Science,Skills Programming Languages Python pandas num...
1,Data Science,Education Details May 2013 May 2017 B E UIT RG...
2,Data Science,Areas Interest Deep Learning Control System De...
3,Data Science,Skills R Python SAP HANA Tableau SAP HANA SQL ...
4,Data Science,Education Details MCA YMCAUST Faridabad Haryan...
...,...,...
957,Testing,Computer Skills Proficient MS office Word Basi...
958,Testing,Willingness accept challenges Positive thinkin...
959,Testing,PERSONAL SKILLS Quick learner Eagerness learn ...
960,Testing,COMPUTER SKILLS SOFTWARE KNOWLEDGE MS Power Po...


In [5]:
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame
# Splitting into train (70%), validation (15%), and test (15%) sets
train_val, test = train_test_split(df, test_size=0.15, random_state=42)
train, val = train_test_split(train_val, test_size=0.1765, random_state=42)  # 15% of original data for validation

# Displaying the shapes of the resulting sets
print("Train set shape:", train.shape)
print("Validation set shape:", val.shape)
print("Test set shape:", test.shape)


Train set shape: (672, 2)
Validation set shape: (145, 2)
Test set shape: (145, 2)


In [80]:
df_train = train.reset_index(drop = True)
df_train

,Category,Cleaned_Resume
0,DevOps Engineer,Software Proficiency Languages Basics C SQL PL...
1,Database,Education Details May 2011 May 2014 Bachelor s...
2,Testing,Willingness accept challenges Positive thinkin...
3,DevOps Engineer,Software Proficiency Languages Basics C SQL PL...
4,Web Designing,SKILLS Languages C Basic JAVA Basic Web Techno...
...,...,...
667,Health and fitness,Education Details May 2014 Diploma Nutrition E...
668,DevOps Engineer,CORE COMPETENCIES Ant Maven GIT Bitbucket Jenk...
669,Java Developer,Education Details August 2010 May 2017 Electro...
670,DevOps Engineer,Software Proficiency Languages Basics C SQL PL...


In [81]:
# Function to convert DataFrame to JSONL
def dataframe_to_jsonl(df, output_file):
    with open(output_file, 'w') as f:
        for index, row in df.iterrows():
            user_text = row['Cleaned_Resume']
            bot_text = row['Category']
            
            conversation = {"text" : f"<user>: {user_text} <bot>: {bot_text}"}
            json.dump(conversation, f)  # Write JSON object to the file
            f.write('\n')  # Add a newline after each JSON object
        

# Convert DataFrame to JSONL file
dataframe_to_jsonl(df_train, 'output.jsonl')


In [82]:
resp = together.Files.check(file="output.jsonl")
print(resp)

{'is_check_passed': True, 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model', 'file_present': 'File found', 'file_size': 'File size 0.002 GB', 'num_samples': 672}


In [83]:
together.Files.check(file="output.jsonl")

{'is_check_passed': True,
 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model',
 'file_present': 'File found',
 'file_size': 'File size 0.002 GB',
 'num_samples': 672}

In [84]:
resp = together.Files.upload(file="output.jsonl")
file_id = resp["id"]
print(file_id)

Uploading output.jsonl: 100%|██████████| 1.67M/1.67M [00:07<00:00, 236kB/s] 

file-81117066-20f7-4bd9-876d-b5d73ed540e0


In [85]:
import together
import os


together.api_key = "3aa940ae895a17febc136d71f381320187e4a727781bb83212df669bc850571d"


resp = together.Finetune.create(
    training_file= 'file-81117066-20f7-4bd9-876d-b5d73ed540e0',
    model = 'mistralai/Mixtral-8x7B-Instruct-v0.1',
    n_epochs = 2,
    n_checkpoints= 1,
    batch_size= 4,
    learning_rate= 1e-5,
    suffix= 'mixtral_job_title_classifier',
    wandb_api_key= '00a7194bab22bc81b8b8f2a9e88f5269441f73d3'
    
)

fine_tune_id = resp['id']
print(resp)

{'training_file': 'file-81117066-20f7-4bd9-876d-b5d73ed540e0', 'validation_file': '', 'model_output_name': 'deshwalkrittika@gmail.com/Mixtral-8x7B-Instruct-v0.1-mixtral_job_title_classifier-2024-03-17-20-17-34', 'model_output_path': 's3://together-dev/finetune/65f74be4d2b96225b1ec1eac/deshwalkrittika@gmail.com/Mixtral-8x7B-Instruct-v0.1-mixtral_job_title_classifier-2024-03-17-20-17-34/ft-aeb464f3-f355-4c02-a6c8-a6a45e70a523', 'Suffix': 'mixtral_job_title_classifier', 'model': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'n_epochs': 2, 'n_checkpoints': 1, 'batch_size': 16, 'learning_rate': 1e-05, 'eval_steps': 0, 'user_id': '65f74be4d2b96225b1ec1eac', 'lora': False, 'lora_r': 8, 'lora_alpha': 8, 'lora_dropout': 0, 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2024-03-17T20:17:34.747Z', 'updated_at': '2024-03-17T20:17:34.747Z', 'status': 'pending', 'owner_address': '0xafda61f7ca707a86dcc21a0cdfc4ef2d9a8cc98e', 'id': 'ft-aeb464f3-f355-4c02-

In [22]:
together.Models.start("deshwalkrittika@gmail.com/Mixtral-8x7B-Instruct-v0.1-mixtral_job_title_classifier-2024-03-17-20-17-34")

{'success': True,
 'value': '391aeea9792c789099b22fea0ab382d0f42ed2258b578c13fcd0e4ebc7dcbec2-0b25c51428a3f01a659bfd411651953e6c7c71af27c33541ed9ae2cd42436d50',
 'wasAlreadyEnabled': True}

In [24]:
# replace this name with the name of your newly finetuned model
new_model_name = 'deshwalkrittika@gmail.com/Mixtral-8x7B-Instruct-v0.1-mixtral_job_title_classifier-2024-03-17-20-17-34'

model_list = together.Models.list()

print(f"{len(model_list)} models available")

available_model_names = [model_dict['name'] for model_dict in model_list]

new_model_name in available_model_names

165 models available


True

In [42]:
# deploy your newly finetuned model
print(new_model_name)
together.Models.start(new_model_name)

deshwalkrittika@gmail.com/Mixtral-8x7B-Instruct-v0.1-mixtral_job_title_classifier-2024-03-17-20-17-34


{'success': True,
 'value': '391aeea9792c789099b22fea0ab382d0f42ed2258b578c13fcd0e4ebc7dcbec2-0b25c51428a3f01a659bfd411651953e6c7c71af27c33541ed9ae2cd42436d50',
 'wasAlreadyEnabled': True}

In [139]:
TOGETHER_API_KEY = "3aa940ae895a17febc136d71f381320187e4a727781bb83212df669bc850571d"
together.api_key = TOGETHER_API_KEY

In [140]:
import together
import openai

together.api_key = TOGETHER_API_KEY
client = openai.OpenAI(api_key=together_api_key, base_url="https://api.together.xyz/v1")

# deploy your newly finetuned model
print(new_model_name)
together.Models.start(new_model_name)

# generate response
response = together.Complete.create(prompt="List places to visit in San Francisco.",  model= new_model_name)

print(response)

deshwalkrittika@gmail.com/Mixtral-8x7B-Instruct-v0.1-mixtral_job_title_classifier-2024-03-17-20-17-34


In [49]:
val = val.reset_index(drop = True)
test = test.reset_index(drop = True)

In [99]:
test.head()

,Category,Cleaned_Resume
0,Java Developer,TECHNICAL SKILLS Programming Languages Java Se...
1,Java Developer,TECHNICALSKILLS SpringMVC Hibernate JDBC Java ...
2,Java Developer,Education Details January 2013 Master Engineer...
3,Hadoop,Skill Set Hadoop Map Reduce HDFS Hive Sqoop ja...
4,Health and fitness,Education Details January 1992 January 2003 Fi...


In [135]:
train =train.reset_index(drop = True)

In [138]:
import time

job_titles = list(train['Category'].unique())
predicted_titles = []
for idx in range(0, len(train)):
    print(f"Processing title {idx + 1}")
    resume = train['Cleaned_Resume'].iloc[idx]
    prompt = f"You are a skilled HR professional, please predict a single job title based on the following resume: {resume}. Strictly return only the single job title and not any details about the resume"
    
    # generate response
    response = together.Complete.create(prompt= prompt,  model= new_model_name, temperature = 0.1, max_tokens = 50)

    output = response['output']['choices'][0]['text']
    predicted_titles.append(output)
    
    

Processing title 1


HTTPError: 503 Server Error: Service Unavailable for url: https://api.together.xyz/api/inference

In [ ]:
predicted_titles

In [ ]:
train['Predicted_Category'] = predicted_titles

In [132]:
# VAL

In [75]:
import time

job_titles = list(val['Category'].unique())
predicted_titles = []
for idx in range(0, len(val)):
    print(f"Processing title {idx + 1}")
    resume = val['Cleaned_Resume'].iloc[idx]
    prompt = f"You are a skilled HR professional, please predict a single job title based on the following resume: {resume}. Strictly return only the single job title and not any details about the resume"
    
    # generate response
    response = together.Complete.create(prompt= prompt,  model= new_model_name, temperature = 0.1, max_tokens = 50)

    output = response['output']['choices'][0]['text']
    predicted_titles.append(output)
    
    time.sleep(3)

Processing title 1
Processing title 2
Processing title 3
Processing title 4
Processing title 5
Processing title 6
Processing title 7
Processing title 8
Processing title 9
Processing title 10
Processing title 11
Processing title 12
Processing title 13
Processing title 14
Processing title 15
Processing title 16
Processing title 17
Processing title 18
Processing title 19
Processing title 20
Processing title 21
Processing title 22
Processing title 23
Processing title 24
Processing title 25
Processing title 26
Processing title 27
Processing title 28
Processing title 29
Processing title 30
Processing title 31
Processing title 32
Processing title 33
Processing title 34
Processing title 35
Processing title 36
Processing title 37
Processing title 38
Processing title 39
Processing title 40
Processing title 41
Processing title 42
Processing title 43
Processing title 44
Processing title 45
Processing title 46
Processing title 47
Processing title 48
Processing title 49
Processing title 50
Processin

In [76]:
predicted_titles

[".\n\nBased on the provided resume, the job title that best fits the candidate's experience and skills is Senior Project Manager or Delivery Manager. The candidate has extensive experience in project management, ITIL management, process improvements, staffing,",
 '. Project Manager',
 '.\n\nBased on the provided resume, the job title would be "Java Developer".',
 '. The job title is: Painting Lecturer',
 '.\n\nBased on the provided resume, a suitable job title could be "CNC Programmer/Operator". The candidate has experience with CNC programming using Fanuc Series milling, as well as experience working with CNC machines. Additionally',
 '.\n\nBased on the provided resume, the job title for this individual would be "Automation Test Lead". This is because the majority of their skills and experience are in the area of automation testing, including experience with various automation tools such',
 '. Project Manager',
 '.\n\nBased on the provided resume, the job title for this individual wo

In [77]:
val['Predicted_Category'] = predicted_titles

In [78]:
val.head(3)

,Category,Cleaned_Resume,Predicted_Category
0,PMO,AREA EXPERTISE PROFILE Around 10 plus years pr...,".\n\nBased on the provided resume, the job tit..."
1,PMO,CORE COMPETENCIES Maintain processes ensure pr...,. Project Manager
2,Java Developer,Skills Language Java Operating System Windows ...,".\n\nBased on the provided resume, the job tit..."


In [82]:

def predict_job_titles(job_title, predicted_job_title,job_titles):
    predicted_titles = []
    context_data = "You are a skilled HR professional who is designated to resolve mismatch in original and predicted job title. For e.g if job_title = 'Data Science' and predicted_job_title = 'Data Scientist', then since both of them are similar, you should return 'Data Science'. Similarly, if job_title = 'Mechanical Engineer' and predicted_job_title = 'Mechanical Designer Engineer', you should return 'Mechanical Engineer' "
    for idx in range(0, len(val)):
        print(f"Processing title {idx + 1}")
        
        prompt = f"Please compare job_title : {job_title[idx]} and predicted job title : {predicted_job_title[idx]}. If the predicted job title is similar, then please predict the corresponding original original job title."
        # Check if predicted_job_title is similar to job_title
        if predicted_job_title[idx].lower() == job_title[idx].lower():
            classified_job_title = job_title[idx]  # Use original job_title
        else:
            # Send the request to the model
            response = client.chat.completions.create(
                messages=[{"role": "system", "content": context_data}, {"role": "user", "content": prompt}],
                model= new_model_name,
                max_tokens=50,
                temperature=0.1
            )

            # Extracting output
            output = response.choices[0].message.content.strip()
            # Extracting job title from the output
            output_job_title = output.split(":")[-1].strip()
            # Filtering the output based on provided job titles
            classified_job_title = [title for title in job_titles if title.lower() in output_job_title.lower()] or [output_job_title]
            classified_job_title = classified_job_title[0]
            #print("Predicted Job Title:", classified_job_title)

        predicted_titles.append(classified_job_title)
    
    return predicted_titles


In [83]:
val.head()

,Category,Cleaned_Resume,Predicted_Category
0,PMO,AREA EXPERTISE PROFILE Around 10 plus years pr...,".\n\nBased on the provided resume, the job tit..."
1,PMO,CORE COMPETENCIES Maintain processes ensure pr...,. Project Manager
2,Java Developer,Skills Language Java Operating System Windows ...,".\n\nBased on the provided resume, the job tit..."
3,Arts,Operating Systems Windows XP Vista 07Education...,. The job title is: Painting Lecturer
4,Mechanical Engineer,SKILLS Knowledge software computer Auto CAD In...,".\n\nBased on the provided resume, a suitable ..."


In [84]:
# Get unique job titles
job_titles = val['Category'].unique()

# Predict job titles for resumes
predicted_job_titles = predict_job_titles(val['Category'], val['Predicted_Category']  , job_titles)

# Print predicted job titles
print("Predicted Job Titles:", predicted_job_titles)

Processing title 1
Processing title 2
Processing title 3
Processing title 4
Processing title 5
Processing title 6
Processing title 7
Processing title 8
Processing title 9
Processing title 10
Processing title 11
Processing title 12
Processing title 13
Processing title 14
Processing title 15
Processing title 16
Processing title 17
Processing title 18
Processing title 19
Processing title 20
Processing title 21
Processing title 22
Processing title 23
Processing title 24
Processing title 25
Processing title 26
Processing title 27
Processing title 28
Processing title 29
Processing title 30
Processing title 31
Processing title 32
Processing title 33
Processing title 34
Processing title 35
Processing title 36
Processing title 37
Processing title 38
Processing title 39
Processing title 40
Processing title 41
Processing title 42
Processing title 43
Processing title 44
Processing title 45
Processing title 46
Processing title 47
Processing title 48
Processing title 49
Processing title 50
Processin

In [87]:
val['LLM_Category'] = predicted_job_titles

In [98]:
val.sample(20)

,Category,Cleaned_Resume,Predicted_Category,LLM_Category
71,Blockchain,SOFTWARE SKILLS Languages C C java Operating S...,.\n\nBlockchain Engineer,Blockchain
58,ETL Developer,Technical Summary Knowledge Informatica Power ...,".\n\nBased on the provided resume, the job tit...",ETL Developer
139,HR,KEY SKILLS Computerized accounting tally Since...,".\n\nBased on the provided resume, the job tit...",HR
105,Electrical Engineering,Education Details July 2016 May 2019 Electrica...,. The job title I predict is: Junior Electrica...,Electrical Engineering
108,Python Developer,Operating Systems Windows Others MS Excel MS O...,".\n\nBased on the provided resume, the job tit...",Python Developer
74,HR,Computer Knowledge Proficient basic use MS off...,". Based on the information provided, a suitabl...",HR
107,Arts,Skills Course Skill Name Board Year Passing Gr...,".\n\nBased on the provided resume, a suitable ...","Based on the information provided, if the pred..."
56,Operations Manager,KEY COMPETENCIES Multi Operations Management P...,".\n\nBased on the provided resume, the single ...",Operations Manager
53,Sales,Education Details Bachelor Bachelor Commerce I...,".\n\nBased on the provided resume, a suitable ...",Sales
30,Java Developer,TECHNICALSKILLS SpringMVC Hibernate JDBC Java ...,". Based on the resume, the job title would be ...",Java Developer


In [91]:
val[val['Category'] == val['LLM_Category']].shape

(137, 4)

In [92]:
val[val['Category'] != val['LLM_Category']].shape

(8, 4)

In [95]:
val.shape

(145, 4)

In [97]:
1 - 8/137

0.9416058394160584

In [111]:
### Test 

In [100]:
import time

job_titles = list(test['Category'].unique())
predicted_titles = []
for idx in range(0, len(test)):
    print(f"Processing title {idx + 1}")
    resume = test['Cleaned_Resume'].iloc[idx]
    prompt = f"You are a skilled HR professional, please predict a single job \
        title based on the following resume: {resume}. Strictly return only the\
            single job title and not any details about the resume"
    
    # generate response
    response = together.Complete.create(prompt= prompt,  model= new_model_name
                                        , temperature = 0.1, max_tokens = 50)

    output = response['output']['choices'][0]['text']
    predicted_titles.append(output)
    
    time.sleep(3)

Processing title 1
Processing title 2
Processing title 3
Processing title 4
Processing title 5
Processing title 6
Processing title 7
Processing title 8
Processing title 9
Processing title 10
Processing title 11
Processing title 12
Processing title 13
Processing title 14
Processing title 15
Processing title 16
Processing title 17
Processing title 18
Processing title 19
Processing title 20
Processing title 21
Processing title 22
Processing title 23
Processing title 24
Processing title 25
Processing title 26
Processing title 27
Processing title 28
Processing title 29
Processing title 30
Processing title 31
Processing title 32
Processing title 33
Processing title 34
Processing title 35
Processing title 36
Processing title 37
Processing title 38
Processing title 39
Processing title 40
Processing title 41
Processing title 42
Processing title 43
Processing title 44
Processing title 45
Processing title 46
Processing title 47
Processing title 48
Processing title 49
Processing title 50
Processin

In [101]:
predicted_titles

['. Based on the resume, the job title that I would predict is Full Stack Java Developer.',
 '. Based on the resume, the job title would be Java Developer.',
 '.\n\nBased on the provided resume, the job title that seems to be the best fit for this candidate is "Java Developer". The candidate has a Master\'s degree in Engineering Information Technology and a Bachelor\'s degree in Engineering Information Technology',
 '.\n\nBased on the provided resume, the job title for this individual would be Hadoop Developer. This is based on the skill set, role, and education details provided in the resume, all of which are centered around Hadoop',
 '.\n\nBased on the provided resume, a suitable job title could be "Fitness Instructor" or "Personal Trainer". The individual has completed a first year science education and holds certifications from the American College of Sports Science and G',
 '.\n\nNetwork Security Engineer',
 '.\n\nBased on the provided resume, the single job title would be "Mechan

In [102]:
test['Predicted_Category'] = predicted_titles

In [103]:
test.head(3)

,Category,Cleaned_Resume,Predicted_Category
0,Java Developer,TECHNICAL SKILLS Programming Languages Java Se...,". Based on the resume, the job title that I wo..."
1,Java Developer,TECHNICALSKILLS SpringMVC Hibernate JDBC Java ...,". Based on the resume, the job title would be ..."
2,Java Developer,Education Details January 2013 Master Engineer...,".\n\nBased on the provided resume, the job tit..."


In [120]:

def predict_job_titles(job_title, predicted_job_title,job_titles):
    predicted_titles = []
    context_data = "You are a skilled HR professional who is designated to resolve mismatch in original and predicted job title. \
        For e.g if job_title = 'Data Science' and predicted_job_title = 'Data Scientist', then since both of them are similar, you \
        should return 'Data Science'. Similarly, if job_title = 'Mechanical Engineer' and predicted_job_title = 'Mechanical Designer \
        Engineer', you should return 'Mechanical Engineer' "
    for idx in range(0, len(test)):
        print(f"Processing title {idx + 1}")
        
        prompt = f"Please compare job_title : {job_title[idx]} and predicted job title : {predicted_job_title[idx]}. If the predicted \
        job title is similar, then please predict the corresponding original original job title."
        # Check if predicted_job_title is similar to job_title
        if predicted_job_title[idx].lower() == job_title[idx].lower():
            classified_job_title = job_title[idx]  # Use original job_title
        else:
            # Send the request to the model
            response = client.chat.completions.create(
                messages=[{"role": "system", "content": context_data}, {"role": "user", "content": prompt}],
                model= new_model_name,
                max_tokens=50,
                temperature=0.1
            )

            # Extracting output
            output = response.choices[0].message.content.strip()
            # Extracting job title from the output
            output_job_title = output.split(":")[-1].strip()
            # Filtering the output based on provided job titles
            classified_job_title = [title for title in job_titles if title.lower() in output_job_title.lower()] or [output_job_title]
            classified_job_title = classified_job_title[0]
            #print("Predicted Job Title:", classified_job_title)

        predicted_titles.append(classified_job_title)
    
    return predicted_titles


In [105]:
test.head()

,Category,Cleaned_Resume,Predicted_Category
0,Java Developer,TECHNICAL SKILLS Programming Languages Java Se...,". Based on the resume, the job title that I wo..."
1,Java Developer,TECHNICALSKILLS SpringMVC Hibernate JDBC Java ...,". Based on the resume, the job title would be ..."
2,Java Developer,Education Details January 2013 Master Engineer...,".\n\nBased on the provided resume, the job tit..."
3,Hadoop,Skill Set Hadoop Map Reduce HDFS Hive Sqoop ja...,".\n\nBased on the provided resume, the job tit..."
4,Health and fitness,Education Details January 1992 January 2003 Fi...,".\n\nBased on the provided resume, a suitable ..."


In [106]:
# Get unique job titles
job_titles = test['Category'].unique()

# Predict job titles for resumes
predicted_job_titles = predict_job_titles(test['Category'], test['Predicted_Category']  , job_titles)

# Print predicted job titles
print("Predicted Job Titles:", predicted_job_titles)

Processing title 1
Processing title 2
Processing title 3
Processing title 4
Processing title 5
Processing title 6
Processing title 7
Processing title 8
Processing title 9
Processing title 10
Processing title 11
Processing title 12
Processing title 13
Processing title 14
Processing title 15
Processing title 16
Processing title 17
Processing title 18
Processing title 19
Processing title 20
Processing title 21
Processing title 22
Processing title 23
Processing title 24
Processing title 25
Processing title 26
Processing title 27
Processing title 28
Processing title 29
Processing title 30
Processing title 31
Processing title 32
Processing title 33
Processing title 34
Processing title 35
Processing title 36
Processing title 37
Processing title 38
Processing title 39
Processing title 40
Processing title 41
Processing title 42
Processing title 43
Processing title 44
Processing title 45
Processing title 46
Processing title 47
Processing title 48
Processing title 49
Processing title 50
Processin

In [114]:
test['LLM_Category'] = predicted_job_titles

In [115]:
test.sample(20)

,Category,Cleaned_Resume,Predicted_Category,LLM_Category
98,Operations Manager,SKILLS Well versed MS Office Internet Applicat...,".\n\nBased on the provided resume, the job tit...",Operations Manager
56,Arts,Skills Course Skill Name Board Year Passing Gr...,".\n\nBased on the provided resume, a suitable ...","Based on the information provided, if the pred..."
42,DevOps Engineer,Software Proficiency Languages Basics C SQL PL...,".\n\nBased on the provided resume, the job tit...",DevOps Engineer
70,ETL Developer,Education Details January 2015 Bachelor Engine...,. ETL Developer,ETL Developer
3,Hadoop,Skill Set Hadoop Map Reduce HDFS Hive Sqoop ja...,".\n\nBased on the provided resume, the job tit...",Hadoop
54,Java Developer,Skills Language Java Operating System Windows ...,".\n\nBased on the provided resume, the job tit...",Java Developer
48,Java Developer,TECHNICAL SKILLS Skills Java SQL PL SQL C C Bo...,".\n\nBased on the provided resume, the job tit...",Java Developer
51,Sales,Skill Sets Multi tasking Collaborative Optimis...,.,Sales
45,Testing,Skill Set OS Windows XP 7 8 8 1 10 Database MY...,. Junior Software Tester,Testing
107,PMO,CORE COMPETENCIES Maintain processes ensure pr...,".\n\nBased on the provided resume, the job tit...",PMO


In [116]:
test[test['Category'] == test['LLM_Category']].shape

(132, 4)

In [123]:
test[test['Category'] != test['LLM_Category']].to_csv('test_fine_tune.csv')

In [124]:
test.shape

(145, 4)

In [125]:
132/145

0.9103448275862069

In [126]:
145-132

13

In [127]:
13+8

21

In [128]:
21/(145+145)

0.07241379310344828

In [131]:
1 -21/290

0.9275862068965517